In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'
if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

# area_sum = 19.925158940748766


Entered /tera04/zhwei/xionghui/bedrock/run/500/


In [3]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    # ('Sbedrock', 'Sbedrock'),
    ('Dbedrock_Frequency', 'Dbedrock'),

    ('Sbedrock', 'Band1'),
    # ('Dbedrock_Frequency', 'Band1'),
    # ('Dbedrock_Frequency_bk2', 'Band1'),

    ('Area','area'),
    ('Koppen', 'Band1'),
    ('IGBP', 'LC', 0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


area_sum = df['Area'].sum()/1e12

# df = df.dropna()
# numeric_columns = df.select_dtypes(include=['float32']).columns
# df[numeric_columns] = df[numeric_columns].round(1)

df = df[df['mask1234'] == 1]
area_DF_sum = df['Area'].sum()/1e12

d_f = (df.groupby('Dbedrock_Frequency')['Area'].sum()/1e12)
d_f2 = (df.groupby('Dbedrock_Frequency')['Area'].sum()/1e12)

# print(area_sum)
print(area_DF_sum)
print(d_f)
print(d_f2)
print(d_f/area_DF_sum)

df.drop(labels='mask1234',axis=1,inplace=True)
with open('csv/Global_DF.csv','w') as f:
    df.to_csv(f)
df

24.831799051794818
Dbedrock_Frequency
1.0     9.738562
4.0    15.093237
Name: Area, dtype: float64
Dbedrock_Frequency
1.0     9.738562
4.0    15.093237
Name: Area, dtype: float64
Dbedrock_Frequency
1.0    0.392181
4.0    0.607819
Name: Area, dtype: float64


,lat,lon,Dbedrock_Frequency,Sbedrock,Area,Koppen,IGBP
708161369,-55.847915,-67.627077,1.0,0.236460,120515.621801,29,1
708161370,-55.847915,-67.622911,1.0,0.279453,120515.621801,29,1
708161371,-55.847915,-67.618744,1.0,0.322446,120515.621801,29,1
708161372,-55.847915,-67.614577,1.0,0.365440,120515.621801,29,4
708161373,-55.847915,-67.610411,1.0,0.408433,120515.621801,29,4
...,...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,1.0,39.301605,26309.225697,29,1
3586500783,82.960426,-26.735409,1.0,39.884819,26309.225697,29,1
3586500784,82.960426,-26.731242,1.0,40.468037,26309.225697,29,1
3586500785,82.960426,-26.727075,1.0,41.051250,26309.225697,29,1


In [9]:
df = pd.read_csv(f'{data_path}csv/Global_DF.csv')

In [11]:
print(df[(df['lat']>-23.5) & (df['lat']<23.5)]['Area'].sum()/1e12)
print(df[(df['lat']>50) & (df['lat']<66.5)]['Area'].sum()/1e12)


11.327967015945639
6.009600120990329


In [6]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    # ('Sbedrock', 'Sbedrock'),
    ('Sbedrock', 'Band1'),
    ('Area', 'area'),

    ('Aboveground', 'Band1'),
    ('Belowground', 'Band1'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df['Aboveground_sum'] = df['Aboveground']*df['Area']
df['Belowground_sum'] = df['Belowground']*df['Area']

ag_sum = df['Aboveground_sum'].sum()
bg_sum = df['Belowground_sum'].sum()

# df = df.dropna()
df = df[df['mask1234'] == 1]

ag_mask = df['Aboveground_sum'].sum()
bg_mask = df['Belowground_sum'].sum()

print(ag_mask/ag_sum)
print(bg_mask/bg_sum)

df.drop(labels='mask1234',axis=1,inplace=True)
with open('csv/Global_biome.csv','w') as f:
    df.to_csv(f)
df

0.3824037301054457
0.31885264416590703


,lat,lon,Sbedrock,Area,Aboveground,Belowground,Aboveground_sum,Belowground_sum
708161369,-55.847915,-67.627077,0.236460,120515.621801,0.0,0.0,0.000000e+00,0.000000e+00
708161370,-55.847915,-67.622911,0.279453,120515.621801,759.0,218.0,9.147136e+07,2.627241e+07
708161371,-55.847915,-67.618744,0.322446,120515.621801,571.0,188.0,6.881442e+07,2.265694e+07
708161372,-55.847915,-67.614577,0.365440,120515.621801,1052.0,281.0,1.267824e+08,3.386489e+07
708161373,-55.847915,-67.610411,0.408433,120515.621801,989.0,271.0,1.191899e+08,3.265973e+07
...,...,...,...,...,...,...,...,...
3586500782,82.960426,-26.739575,39.301605,26309.225697,2.0,8.0,5.261845e+04,2.104738e+05
3586500783,82.960426,-26.735409,39.884819,26309.225697,2.0,8.0,5.261845e+04,2.104738e+05
3586500784,82.960426,-26.731242,40.468037,26309.225697,2.0,8.0,5.261845e+04,2.104738e+05
3586500785,82.960426,-26.727075,41.051250,26309.225697,2.0,10.0,5.261845e+04,2.630923e+05


In [ ]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Band1'),
    ('Sr', 'Band1'),
    # ('Sbedrock', 'Sbedrock'),
    # ('Sr', 'Sr'),
    ('Dbedrock_Frequency', 'Dbedrock'),
    ('PR_mean', 'tp', 0),
    ('ET_mean', 'et', 0),

    ('Area','area')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  

    # print(f"File: {file}, Variable: {variable_name}, Index: {index}")
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

# df = df.dropna()
# numeric_columns = df.select_dtypes(include=['float32']).columns
# df[numeric_columns] = df[numeric_columns].round(1)

df = df[df['mask1234'] == 1]
with open('csv/Global_PR.csv','w') as f:
    df.to_csv(f)
df

,lat,lon,mask1234,Sbedrock,Sr,Dbedrock_Frequency,PR_mean,ET_mean,Area
708161369,-55.847917,-67.627083,1.0,0.236460,0.236460,1.0,692.439941,2.555584,120515.621801
708161370,-55.847917,-67.622916,1.0,0.279453,0.279453,1.0,691.950134,3.020087,120515.621801
708161371,-55.847917,-67.618749,1.0,0.322446,0.322446,1.0,691.460266,3.484589,120515.621801
708161372,-55.847917,-67.614583,1.0,0.365440,0.365440,1.0,690.970398,3.949090,120515.621801
708161373,-55.847917,-67.610416,1.0,0.408433,0.408433,1.0,690.480591,4.413592,120515.621801
...,...,...,...,...,...,...,...,...,...
3586500782,82.960414,-26.739583,1.0,39.301605,39.301605,1.0,125.175171,107.267189,26309.225697
3586500783,82.960414,-26.735417,1.0,39.884819,39.884819,1.0,125.231812,107.503296,26309.225697
3586500784,82.960414,-26.731250,1.0,40.468037,40.468037,1.0,125.288445,107.739403,26309.225697
3586500785,82.960414,-26.727083,1.0,41.051250,41.051250,1.0,125.345085,107.975510,26309.225697


: 

In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Band1'),
    # ('Sbedrock', 'Sbedrock'),
    ('Dbedrock_Frequency', 'Dbedrock'),
    ('Area','area'),
    
    ('P/Sbedrock_div_Sr', 'Sbedrock'),
    ('P/Sbedrock_div_ET_mean', 'Sbedrock'),
    ('P/Sbedrock_div_PR_mean', 'Sbedrock'),
    # ('P/ET_mean_div_PR_mean', 'et'),
    # ('P/ET_mean_sub_Sbedrock_div_PR_mean', 'et'),
    # ('P/Q_mean_div_PR_mean', 'tp'),
    # ('P/PET_div_PR_mean', 'Band1')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['mask1234'] == 1]

dbedrcok_frequency = df.groupby('Dbedrock_Frequency')['Area'].sum()/1e12
print(dbedrcok_frequency)


df.drop(labels='mask1234',axis=1,inplace=True)
with open('csv/Global_index.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Sr', 'Sr'),
    ('Ssoil', 'Band1'),
    ('Area','area')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  

    # print(f"File: {file}, Variable: {variable_name}, Index: {index}")
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
# df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]

# print((df.groupby('D_FM_median')['Area'].sum()/1e12)/16.092963536896)
area_sum = df['Area'].sum()/1e12
sbedrock_mean = df['Sbedrock'].mean()
sr_mean = df['Sr'].mean()
ssoil_mean = df['Ssoil'].mean()
print(area_sum)
print(sbedrock_mean)
print(sr_mean)
print(ssoil_mean)


df.drop(labels='mask1234',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)

with open('csv/Global_S.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
# df = pd.DataFrame()

# with nc.Dataset('mask1234.nc4') as dataset:
#     lat = dataset['lat'][:].flatten()
#     lon = dataset['lon'][:].flatten()
#     df['lat'] = np.repeat(lat, len(lon))
#     df['lon'] = np.tile(lon, len(lat))

# file_variable_list = [
#     ('mask1234', 'Band1'),
#     ('Sbedrock', 'Sbedrock'),

#     ('S_FD', 'FD'),
#     ('S_FY', 'FD'),
#     # ('D_FD_median', 'FD'),
#     # ('D_FM_median', 'FD'),
#     ('D_FD_mean', 'FD'),
#     ('D_FM_mean', 'FD'),

#     ('S_Duration', 'Duration'),
#     # ('D_Duration_median', 'Duration'),
#     ('D_Duration_mean', 'Duration'),

#     ('Area', 'area')
# ]

# for entry in file_variable_list:
#     file = entry[0]
#     variable_name = entry[1]  
#     index = entry[2:] if len(entry) > 2 else None  
#     if index:
#         df[file] = load_and_flatten_data(file, variable_name, index[0])
#     else:
#         df[file] = load_and_flatten_data(file, variable_name)


# df = df.dropna()
# df = df[df['Sbedrock'] > 0]
# df = df[df['mask1234'] == 1]

# df['S_FD'] = df['S_FD'].round().astype(int)
# df['S_FY'] = df['S_FY'].round().astype(int)
# df['D_FD_mean'] = df['D_FD_mean'].round().astype(int)
# df['D_FM_mean'] = df['D_FM_mean'].round().astype(int)
# df['S_Duration'] = df['S_Duration'].round().astype(int)
# df['D_Duration_mean'] = df['D_Duration_mean'].round().astype(int)


# s_fd = (df.groupby('S_FD')['Area'].sum()/1e12)/area_sum
# s_fy = (df.groupby('S_FY')['Area'].sum()/1e12)/area_sum
# d_fd_mean = (df.groupby('D_FD_mean')['Area'].sum()/1e12)/area_sum
# d_fm_mean = (df.groupby('D_FM_mean')['Area'].sum()/1e12)/area_sum
# s_duration = (df.groupby('S_Duration')['Area'].sum()/1e12)/area_sum
# d_duration_mean = (df.groupby('D_Duration_mean')['Area'].sum()/1e12)/area_sum

# # print(s_fd)
# print(s_fy)
# # print(d_fd_mean)
# print(d_fm_mean)
# # print(s_duration)
# # print(d_duration_mean)

# print(s_fd[s_fd.index <= 1000].sum())
# print(s_fy[s_fy.index <= 1].sum())
# print(d_fd_mean[d_fd_mean.index <= 50].sum())
# print(d_fm_mean[d_fm_mean.index <= 1].sum())
# print(s_duration[s_duration.index >= 365].sum())
# print(d_duration_mean[d_duration_mean.index >= 30].sum())

# print((df['Area'].sum()/1e12)/area_sum)

# df.drop(labels='mask1234',axis=1,inplace=True)
# # df.drop(labels='Area',axis=1,inplace=True)
# with open('csv/Global_FDFMFY.csv','w') as f:
#     df.to_csv(f)
# df

In [ ]:
# df = pd.DataFrame()

# with nc.Dataset('mask1234.nc4') as dataset:
#     lat = dataset['lat'][:].flatten()
#     lon = dataset['lon'][:].flatten()
#     df['lat'] = np.repeat(lat, len(lon))
#     df['lon'] = np.tile(lon, len(lat))

# file_variable_list = [
#     ('mask1234', 'Band1'),
#     ('Sbedrock', 'Sbedrock'),
#     ('Area', 'area'),

#     ('Drought_Period_max_sub_median', 'Drought_Period', 0),
#     ('Drought_Period_timmax', 'Drought_Period', 0),
#     ('Drought_Period_yearmax_median', 'Dbedrock'),
#     ('Drought_Period_yearmax_timmean', 'Drought_Period', 0),
# ]

# for entry in file_variable_list:
#     file = entry[0]
#     variable_name = entry[1]  
#     index = entry[2:] if len(entry) > 2 else None  
#     if index:
#         df[file] = load_and_flatten_data(file, variable_name, index[0])
#     else:
#         df[file] = load_and_flatten_data(file, variable_name)


# df = df.dropna()
# df = df[df['Sbedrock'] > 0]
# df = df[df['mask1234'] == 1]

# df['Drought_Period_max_sub_median'] = df['Drought_Period_max_sub_median'].round().astype(int)
# df['Drought_Period_timmax'] = df['Drought_Period_timmax'].round().astype(int)
# df['Drought_Period_yearmax_median'] = df['Drought_Period_yearmax_median'].round().astype(int)
# df['Drought_Period_yearmax_timmean'] = df['Drought_Period_yearmax_timmean'].round().astype(int)

# dpmsm = (df['Drought_Period_max_sub_median'].mean())
# dpt = (df['Drought_Period_timmax'].mean())
# dpym = (df['Drought_Period_yearmax_median'].mean())
# dpyt = (df['Drought_Period_yearmax_timmean'].mean())

# print(dpmsm)
# print(dpt)
# print(dpym)
# print(dpyt)


# df.drop(labels='mask1234',axis=1,inplace=True)
# with open('csv/Global_Drought.csv','w') as f:
#     df.to_csv(f)
# df

In [ ]:
# print(s_fd[s_fd.index <= 1000].sum())
# print(s_fy[s_fy.index <= 1].sum())
# print(d_fd_mean[d_fd_mean.index <= 50].sum())
# print(d_fm_mean[d_fm_mean.index <= 1].sum())
# print(s_duration[s_duration.index > 365].sum())
# print(d_duration_mean[d_duration_mean.index > 30].sum())

In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area','area'),

    ('Dbedrock_median', 'Dbedrock'),
    ('D/Dbedrock_2011', 'Dbedrock'),
    ('D/Dbedrock_2014', 'Dbedrock'),
    ('D/Dbedrock_2015', 'Dbedrock'),
    ('D/Dbedrock_2017', 'Dbedrock')
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df = df.dropna()
df = df[df['mask1234'] == 1]
# df = df[df['Sbedrock'] > 0]

dbedrock_median = df[df['Dbedrock_median'] > 0]['Dbedrock_median'].mean()
dbedrock_2011 = df[df['D/Dbedrock_2011'] > 0]['D/Dbedrock_2011'].mean()
dbedrock_2014 = df[df['D/Dbedrock_2014'] > 0]['D/Dbedrock_2014'].mean()
dbedrock_2015 = df[df['D/Dbedrock_2015'] > 0]['D/Dbedrock_2015'].mean()
dbedrock_2017 = df[df['D/Dbedrock_2017'] > 0]['D/Dbedrock_2017'].mean()

print(dbedrock_median)
print(dbedrock_2011)
print(dbedrock_2014)
print(dbedrock_2015)
print(dbedrock_2017)

df.drop(labels='Sbedrock',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask1234',axis=1,inplace=True)
with open('csv/Global_Db.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),
    ('Koppen', 'Band1'),
    ('IGBP', 'LC', 0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

df = df.dropna()
# df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]

df['Koppen'] = df['Koppen'].round().astype(int)
df['IGBP'] = df['IGBP'].round().astype(int)

koppen_sum =  (df.groupby('Koppen')['Area'].sum()/1e12)/area_sum
igbp_sum =  (df.groupby('IGBP')['Area'].sum()/1e12)/area_sum

print(koppen_sum)
print(igbp_sum)

df.drop(labels='mask1234',axis=1,inplace=True)
# df.drop(labels='Area',axis=1,inplace=True)
with open('csv/Global_igbp_koppen.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('DTB', 'Band1'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
# df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]

print(df['Area'].sum()/1e12)

# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask1234',axis=1,inplace=True)
df.drop(labels='Sbedrock',axis=1,inplace=True)

with open('csv/Global_DTB.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
# print(area_sum)
# print(sbedrock_mean)
# print(sr_mean)
# print(ssoil_mean)

# print(area_DF_sum)

# print(s_fd)
# print(s_fy)
# print(d_fd_mean)
# print(d_fm_mean)
# print(s_duration)
# print(d_duration_mean)

# print(s_fd[s_fd.index <= 1000].sum())
# print(s_fy[s_fy.index <= 1].sum())
# print(d_fd_mean[d_fd_mean.index <= 50].sum())
# print(d_fm_mean[d_fm_mean.index <= 1].sum())
# print(s_duration[s_duration.index >= 365].sum())
# print(d_duration_mean[d_duration_mean.index >= 30].sum())

# print(dbedrcok_frequency)

# print(dbedrock_median)
# print(dbedrock_2011)
# print(dbedrock_2014)
# print(dbedrock_2015)
# print(dbedrock_2017)

# print(koppen_sum)
# print(igbp_sum)

# print(ag_mask/ag_sum)
# print(bg_mask/bg_sum)